In [1]:
import serial
from matplotlib import pyplot
from PIL import Image
from inception_resnet_v1 import *
model = InceptionResNetV1()
from matplotlib.patches import Rectangle
from matplotlib.patches import Circle
from mtcnn.mtcnn import MTCNN
K.set_image_data_format('channels_last')
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import smtplib, ssl, email
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import imghdr
from twilio.rest import Client
from email.message import EmailMessage
import PIL
import cv2
from time import sleep

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
face6 = extract_faces(r"C:\Users\sam\Downloads\Telegram Desktop\aman\aman.jpg")
face7 = extract_faces(r"C:\Users\sam\Downloads\Telegram Desktop\aman\aman2.jpg")



In [8]:
database = {}
database["name"]  = get_embedding(model, face6)
database["name2"] = get_embedding(model, face7)

In [9]:
def extract_faces(filename,required_size=(160, 160)):
  image = Image.open(filename)
  #image = image.transpose(Image.ROTATE_180)
  image = image.convert('RGB')
  pixels = np.asarray(image)
  detector = MTCNN()
  results = detector.detect_faces(pixels)
  if not len(results):
    print(filename, "cant be encoded")
    image = Image.fromarray(pixels)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    return face_array
    pass
  x1, y1, width, height = results[0]['box']
  x1, y1 = abs(x1), abs(y1)
  x2, y2 = x1 + width, y1 + height
  face = pixels[y1:y2, x1:x2]
  image = Image.fromarray(face)
  image = image.resize(required_size)
  face_array = np.asarray(image)
  #database["sam"] = img_to_encoding(face_array, model)
  return face_array

In [6]:
def get_embedding(model, face_pixels):
    face_pixels = face_pixels.astype('float32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    samples = np.expand_dims(face_pixels, axis=0)
    yhat = model.predict(samples)
    return yhat[0]

In [12]:

def who_is_it(database,model):
   
    #encoding =  img_to_encoding(image_path, model)
    key = cv2. waitKey(1)
    webcam = cv2.VideoCapture(0)
    sleep(2)
   # ser = serial.Serial('COM10', 9600) #you put your serial port number on COM10
    while True:
    #rate(25)
        while(ser.inWaiting() <= 10):# recives serial data from arduino 
            data = ser.readline()
            dist = float(data)

        try:
            check, frame = webcam.read() # intiates the camera and capture
            cv2.imshow("Capturing", frame)
            key = cv2.waitKey(1)
            while check: 
                cv2.imwrite(filename='saved_img.jpg', img=frame)
                webcam.release()
                cv2.destroyAllWindows()
                check = False

        except(KeyboardInterrupt):
            print("Turning off camera.")
            webcam.release()
            print("Camera off.")
            print("Program ended.")
            cv2.destroyAllWindows()
        try:
            new = extract_faces('saved_img.jpg')
            encoding = get_embedding(model, new)
        except:
            encoding = get_embedding(model, 'saved_img.jpg')
            pass
       # encoding = get_embedding(model, new)
        min_dist = 100
        for (name, db_enc) in database.items():
            dist = np.linalg.norm(encoding - database[name])# it calculates the euclidean distance of the captured pic and the one in the database
            if dist<min_dist:
                min_dist = dist
                identity = name

        if min_dist > 0.1:
            print("Not in the database.")
            client = Client("AC123644fe5b14bf2153faf868d8a3f39e", "225c1c37c890464c43b5beb7fc7a568e")
            Sender_Email = "sender emails" #needs interner connection
            Reciever_Email = "reviver email"
            Password = 'sender password'
            newMessage = EmailMessage()                         
            newMessage['Subject'] = "anti theft" 
            newMessage['From'] = Sender_Email                   
            newMessage['To'] = Reciever_Email                   
            newMessage.set_content('do you know this person!') 
            with open('saved_img.jpg', 'rb') as f:
                image_data = f.read()
                image_type = imghdr.what(f.name)
                image_name = f.name

            newMessage.add_attachment(image_data, maintype='image', subtype=image_type, filename=image_name)
            with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
                client.messages.create(to="+your number", 
                                  from_="+your twilio account number", 
                                  body="check your email there a person at your door")
                smtp.login(Sender_Email, Password)              
                smtp.send_message(newMessage)

        else:
            print ("it's " + str(identity) + ", the distance is " + str(min_dist))

        return min_dist, identity




In [13]:
while True:
    who_is_it(database, model)

Not in the database.


TwilioRestException: HTTP 400 error: Unable to create record: The 'To' number + is not a valid phone number.